In [1]:
import yfinance as yf
import requests
import json
import re

# 获取公司简况
def get_company_overview(stock_code):
    stock = yf.Ticker(stock_code)
    info = stock.info
    overview = info.get('longBusinessSummary', 'No description available')
    return overview

# 调用Ollama QWQ-32B进行行业分类
def classify_industry(overview):
    url = "http://localhost:11434/api/generate"  # Ollama API端点
    prompt = f"请根据以下公司简介，分析并用简洁的语言回答：\n1. 主要业务类型（科技/制造/金融/能源/消费等）\n2. 经营模式（轻资产/重资产）\n3. 市场地位（垄断/寡头/充分竞争）\n4. 行业周期性（强周期/弱周期/非周期）\n\n公司简介：{overview}\n\n请用逗号分隔每个维度的结论，例如：科技,轻资产,充分竞争,弱周期"
    payload = {
        "model": "qwq:latest",
        "prompt": prompt,
        "stream": False
    }
    response = requests.post(url, json=payload)
    result = response.json()['response']
    
    # 处理结果，提取最终结论
    # 尝试查找最后一行包含逗号分隔的结论
    lines = result.strip().split('\n')
    for line in reversed(lines):
        # 查找符合"类型,类型,类型,类型"格式的行
        if line.count(',') == 3 and all(part.strip() for part in line.split(',')):
            return line.strip()
    
    # 如果没有找到符合格式的行，尝试使用正则表达式提取
    pattern = r'([^,\n]+,[^,\n]+,[^,\n]+,[^,\n]+)'
    matches = re.findall(pattern, result)
    if matches:
        return matches[-1].strip()
    
    # 如果仍然无法提取，返回原始结果并发出警告
    print("警告：无法提取格式化的行业分类结果")
    return result

def calculate_base_pe(company_type, discount_factor=0.6):
    """
    根据公司类型和折价系数计算基础PE值，使用有限状态机实现。
    
    参数:
    company_type (str): 公司类型，例如"消费,重资产,充分竞争,弱周期"
    discount_factor (float): 最终折价系数，默认0.6
    
    返回:
    float: 调整后的基础PE值
    """
    # 解析公司类型
    try:
        business_type, asset_model, market_status, cycle_type = company_type.split(',')
        business_type = business_type.strip()
        asset_model = asset_model.strip()
        market_status = market_status.strip()
        cycle_type = cycle_type.strip()
    except ValueError:
        raise ValueError("公司类型格式错误，应为'业务类型,经营模式,市场地位,行业周期性'")

    # 定义状态机初始值和调整系数
    base_pe = 0.0  # 初始PE
    state = 0      # 状态机状态

    # 状态转移逻辑
    while state <= 4:
        if state == 0:  # 状态0：设定基础PE（主要业务类型）
            if business_type == "科技":
                base_pe = 25
            elif business_type == "制造":
                base_pe = 15
            elif business_type == "金融":
                base_pe = 12
            elif business_type == "能源":
                base_pe = 15
            elif business_type == "消费":
                base_pe = 20
            else:
                raise ValueError(f"未知业务类型: {business_type}")
            state += 1

        elif state == 1:  # 状态1：经营模式调整
            if asset_model == "轻资产":
                base_pe *= 1.1
            elif asset_model == "重资产":
                base_pe *= 0.9
            else:
                raise ValueError(f"未知经营模式: {asset_model}")
            state += 1

        elif state == 2:  # 状态2：市场地位调整
            if market_status == "垄断":
                base_pe *= 1.1
            elif market_status == "寡头":
                base_pe *= 1.0
            elif market_status == "充分竞争":
                base_pe *= 0.9
            else:
                raise ValueError(f"未知市场地位: {market_status}")
            state += 1

        elif state == 3:  # 状态3：行业周期性调整
            if cycle_type == "非周期":
                base_pe *= 1.1
            elif cycle_type == "弱周期":
                base_pe *= 1.0
            elif cycle_type == "强周期":
                base_pe *= 0.9
            else:
                raise ValueError(f"未知行业周期性: {cycle_type}")
            state += 1

        elif state == 4:  # 状态4：应用折价系数
            base_pe *= discount_factor
            state += 1

    return base_pe

In [2]:
# 示例运行
stock_code = "9987.HK"
# 获取简况
overview = get_company_overview(stock_code)
print(f"公司简介: {overview[:200]}...")  # 显示前200字符
# 行业分类
analysis = classify_industry(overview)
print(f"公司类型: {analysis}")

公司简介: Yum China Holdings, Inc. owns, operates, and franchises restaurants in the People's Republic of China. The company operates through KFC, Pizza Hut, and All Other segments. It operates restaurants unde...
公司类型: 消费,轻资产,寡头,弱周期


In [6]:
base_pe = calculate_base_pe(analysis, discount_factor=0.8)
print(f"基础PE: {base_pe:.2f}")

基础PE: 17.60


In [4]:
# 获取历史净利润增长率
def get_profit_growth_rate(stock_code, years=3):
    stock = yf.Ticker(stock_code)
    
    # 获取年度财务数据
    financials = stock.financials
    
    # 尝试获取净利润数据
    try:
        if 'Net Income' in financials.index:
            net_income = financials.loc['Net Income']
        elif 'Net Income Common Stockholders' in financials.index:
            net_income = financials.loc['Net Income Common Stockholders']
        else:
            # 如果找不到净利润相关指标，返回None
            print(f"无法找到{stock_code}的净利润数据")
            return None
        
        # 计算年度增长率
        growth_rates = []
        for i in range(len(net_income) - 1):
            if net_income.iloc[i+1] != 0:  # 避免除以零
                growth_rate = (net_income.iloc[i] - net_income.iloc[i+1]) / abs(net_income.iloc[i+1])
                growth_rates.append(growth_rate)
        
        # 如果数据不足years年，则使用所有可用数据
        available_years = min(years, len(growth_rates))
        if available_years == 0:
            print(f"没有足够的历史数据计算{stock_code}的增长率")
            return 0.05  # 默认5%增长率
            
        # 计算平均增长率
        avg_growth_rate = sum(growth_rates[:available_years]) / available_years
        return avg_growth_rate
        
    except Exception as e:
        print(f"计算{stock_code}增长率时出错: {e}")
        return 0.05  # 默认5%增长率

# 预测未来PE
def predict_future_pe(base_pe, growth_rate, years=3):
    """
    基于基础PE和增长率预测未来PE
    
    参数:
    base_pe (float): 基础PE值
    growth_rate (float): 年平均净利润增长率
    years (int): 预测年数
    
    返回:
    dict: 包含各年预测PE的字典
    """
    future_pes = {}
    
    # 增长率调整 - 高增长率适当折价，避免过度乐观
    adjusted_growth_rate = growth_rate
    if growth_rate > 0.3:
        adjusted_growth_rate = 0.3 + (growth_rate - 0.3) * 0.5  # 超过30%的部分打五折
    
    # 负增长率调整 - 避免过度悲观
    if growth_rate < -0.2:
        adjusted_growth_rate = -0.2 + (growth_rate + 0.2) * 0.5  # 低于-20%的部分打五折
    
    # 计算各年PE
    for year in range(1, years + 1):
        # PE随增长率变化，高增长对应高PE
        growth_factor = 1 + (adjusted_growth_rate * year)
        # 确保增长因子不会导致PE变为负数
        growth_factor = max(0.5, growth_factor)
        future_pes[f"Year_{year}"] = base_pe * growth_factor
    
    return future_pes

In [7]:
# 获取历史增长率
growth_rate = get_profit_growth_rate(stock_code)
print(f"历史年平均净利润增长率: {growth_rate:.2%}")

# 预测未来PE
future_pes = predict_future_pe(base_pe, growth_rate)
print("\n未来PE预测:")
for year, pe in future_pes.items():
    print(f"{year}: {pe:.2f}")

# 获取当前股价
stock = yf.Ticker(stock_code)
current_price = stock.info.get('currentPrice', None)
if current_price:
    # 获取最近的每股收益
    pe = stock.info.get('trailingPE', None)
    if pe and pe != 0:
        current_pe = pe
        print(f"\n当前股价: {current_price:.2f}, 当前PE: {current_pe:.2f}")
        
        # 估值分析
        avg_future_pe = sum(future_pes.values()) / len(future_pes)
        if current_pe < avg_future_pe * 0.8:
            valuation = "低估"
        elif current_pe > avg_future_pe * 1.2:
            valuation = "高估"
        else:
            valuation = "合理"
        
        print(f"三年平均预期PE: {avg_future_pe:.2f}, 估值状态: {valuation}")
    else:
        print("无法获取每股收益数据")
else:
    print("无法获取当前股价")

历史年平均净利润增长率: 13.97%

未来PE预测:
Year_1: 20.06
Year_2: 22.52
Year_3: 24.98

当前股价: 377.80, 当前PE: 20.86
三年平均预期PE: 22.52, 估值状态: 合理
